In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load the image
image = Image.open('gsm8k.png')

# Display the image
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis('off')
plt.title('GSM8K Image')
plt.show()


: 